In [0]:
#Import libraries
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import json
import os
import re
import joblib
import time
import email
import multiprocessing
from tqdm.notebook import tqdm
import pickle
import random
import datetime
import tensorflow as tf
from tensorflow.keras.layers import Dense,Input,LSTM,Embedding,Dropout,TimeDistributed,BatchNormalization
from tensorflow.keras.models import Model
from nltk.translate import bleu_score

In [2]:
!wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-IN,en-GB;q=0.9,en-US;q=0.8,en;q=0.7" --header="Referer: https://www.kaggle.com/" "https://storage.googleapis.com/kaggle-data-sets/55%2F120%2Fcompressed%2Femails.csv.zip?GoogleAccessId=gcp-kaggle-com@kaggle-161607.iam.gserviceaccount.com&Expires=1592565748&Signature=Y06D08ms9y%2FQscjdubNlfmeWOhAx8Gij27O1C7TfS9wFlDRXthY0jEhq0igZktZpptQIRPK6Ek4rh0uBbpTLxDkOJYQJAr%2Bz608UUr8yCKhXadu0u3VympFlPhusgrjEhvxCfFisw2OeaoYrmSRBSMw%2BS8W6ktCWWiikX2KTOPJLHBy2Aql0AoUHJnT05jgW3PYeWASm%2FFpA0t3DyvrWjWIMPwIqWkfPiUR3FYk%2FKKp7LMlkcFJ5t2v%2FyCQ8Y1Qr4mHk7F5Zg%2FVewxE05CRhdsKavKAD23A961UWnJdHBBWiE1MP2hRDIagKPNUmwasbOVOdVK9UUcYM0Ek0wZGBqg%3D%3D" -c -O '55_120_compressed_emails.csv.zip'

--2020-06-16 11:23:07--  https://storage.googleapis.com/kaggle-data-sets/55%2F120%2Fcompressed%2Femails.csv.zip?GoogleAccessId=gcp-kaggle-com@kaggle-161607.iam.gserviceaccount.com&Expires=1592565748&Signature=Y06D08ms9y%2FQscjdubNlfmeWOhAx8Gij27O1C7TfS9wFlDRXthY0jEhq0igZktZpptQIRPK6Ek4rh0uBbpTLxDkOJYQJAr%2Bz608UUr8yCKhXadu0u3VympFlPhusgrjEhvxCfFisw2OeaoYrmSRBSMw%2BS8W6ktCWWiikX2KTOPJLHBy2Aql0AoUHJnT05jgW3PYeWASm%2FFpA0t3DyvrWjWIMPwIqWkfPiUR3FYk%2FKKp7LMlkcFJ5t2v%2FyCQ8Y1Qr4mHk7F5Zg%2FVewxE05CRhdsKavKAD23A961UWnJdHBBWiE1MP2hRDIagKPNUmwasbOVOdVK9UUcYM0Ek0wZGBqg%3D%3D
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.203.128, 2404:6800:4008:c07::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.203.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 375294957 (358M) [application/zip]
Saving to: ‘55_120_compressed_emails.csv.zip’

55_120_compressed_e 100%[===================>] 357.91M  44.8MB/s    in 9.0s    

2020-06-16 11:

In [3]:
!unzip 55_120_compressed_emails.csv.zip

Archive:  55_120_compressed_emails.csv.zip
  inflating: emails.csv              


<h1>1. Sentence Extraction

<h2>1.1. Read dataset

In [46]:
#Read the dataset
data = pd.read_csv('emails.csv',nrows=50000)
data.head()

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


<h2>1.2. Extract content from email

In [47]:
#Extract messages from email
messages = [email.message_from_string(i) for i in tqdm(data['message'])]

In [0]:
#https://www.kaggle.com/zichen/explore-enron#1.-Loading-and-cleaning-data
#Get email contents
def get_content(text):
    """
    Get content from email
    """
    parts = []
    for part in text.walk():
        if part.get_content_type() == 'text/plain':
            parts.append(part.get_payload())
    return ''.join(parts)

In [49]:
#Get content from email
data['content'] = [get_content(i) for i in tqdm(messages)]

In [50]:
data = data['content'].to_frame()
data.head()

,content
0,Here is our forecast\n\n
1,Traveling to have a business meeting takes the...
2,test successful. way to go!!!
3,"Randy,\n\n Can you send me a schedule of the s..."
4,Let's shoot for Tuesday at 11:45.


<h2>1.3. Preprocess data

In [0]:
#Remove insignificant emails
data = data[~data['content'].str.contains('Outlook Migration Team@ENRON')]
data = data[~data['content'].str.contains('Trade Counts and Volume')]

In [0]:
#https://github.com/JuberGandharv/Building_Email_Smart_Compose_Real_Time_Assisted_Writing/blob/master
#/bahdanau_attention_model_train.ipynb
def get_content_from_emails(sentence):
    
    """
    Returns body(content) of the email
    """
    
    sentences = sentence.split('\n')
    a = ('---------------------- Forwarded by','-----Original Message-----','Subject:','Sincerely,','Sincerely yours,',
         'Thank you,''Thanks.','Best regards,','Gratefully yours,','Best regards,','regards,','Thank you for your assistance.',
         'Thank you for your assistance','Thank you for your help')
    indices = [i for i, s in enumerate(sentences) if any(c in s for c in a)]
    temp_sent = ''

    if len(indices)>1:
        if any(sub_string in sentences[indices[0]] for sub_string in ('---------------------- Forwarded by',
                                                                        '-----Original Message-----')):
            for i in range(1,len(indices),2):
                if i < len(indices)-1:
                    start = indices[i] +1
                    end = indices[i+1] 
                    temp_sent = temp_sent + ' '.join(sentences[start:end])
                else:
                    start = indices[i]+1
                    temp_sent = temp_sent + ' '.join(sentences[start:])
        elif any(sub_string in sentences[indices[0]] for sub_string in ('Subject:')):
            for i in range(0,len(indices),2):
                if i < len(indices)-1:
                    start = indices[i] +1
                    end = indices[i+1] 
                    temp_sent = temp_sent + ' '.join(sentences[start:end])
                else:
                    start = indices[i]+1
                    temp_sent = temp_sent + ' '.join(sentences[start:])

    elif len(indices) == 1:
        if any(sub_string in sentences[indices[0]] for sub_string in ('---------------------- Forwarded by',
                                                                        '-----Original Message-----','Subject:')):
            start = indices[0] + 1
            temp_sent = temp_sent + ' '.join(sentences[start:])
        else:
            stop = indices[0]+1
            temp_sent = temp_sent + ' '.join(sentences[:stop])

    else:
        temp_sent = temp_sent + ' '.join(sentences[:])
    
    return temp_sent

In [53]:
#Extract content from forwarded emails
tqdm.pandas()
data['content'] = data['content'].progress_apply(get_content_from_emails)

<h2>1.4. Get sentences from content

In [0]:
def hasNumbers(text):
    return bool(re.search(r'\d', text))

In [0]:
def check_special_characters(text): 
    regex = re.compile("""[-+=*@_#`"$%^&*[\]\()<>/\|}{~:]""") 
    if(regex.search(text) == None): 
        return True     
    else:
        return False

In [0]:
#Decontraction of text
def decontracted(phrase):
    """
    Returns decontracted phrases
    """
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [0]:
def remove_extra_spaces(text):
    #Conisder only alphabets
    text = re.sub('[^A-Za-z]+', ' ', text)
    return ' '.join(text.split()).strip().lower()

In [0]:
def get_sentences(content):
    sentences_list = []
    for i in content.split('.'):
        #Strip the sentence
        sentence = i.strip()
        #Conisder sentences which have between 3 to 20 words
        if 3<=len(sentence.split())<=20:
            #Check if first letter starts with upper case and rest all words are lower case
            if sentence[0].isupper() and sentence[1:].islower():
                #Drop sentences which have numbers
                if hasNumbers(sentence)==False:
                    #Drop sentences which have special characters
                    if check_special_characters(sentence)==True:
                        #Decontract the phrases
                        sentence = decontracted(sentence)
                        #Split sentence from question mark
                        for j in re.split('(?<=[?]) +',sentence):
                            #Remove extra spaces and append to list
                            sentences_list.append(remove_extra_spaces(j))
    return pd.DataFrame({'sent':sentences_list})

In [0]:
def sentence_df(data_df):
    for i in tqdm(range(data_df.shape[0])):
        if i==0:
            final_df = get_sentences(data_df.content.iloc[0])
        else:
            final_df = pd.concat([final_df,get_sentences(data_df.content.iloc[i])],ignore_index=True)
    final_df = final_df.drop_duplicates()
    return final_df

In [60]:
#Get sentences from content
sentences = sentence_df(data)

In [61]:
#Consider sentences which are in vocabulary
vocab = joblib.load('vocabulary')
indices = []
for i in tqdm(range(sentences.shape[0])):
    for j in sentences.sent.iloc[i].split():
        if j not in vocab:
            break
    else:
        indices.append(i)
sentences = sentences.iloc[indices].drop_duplicates()

In [62]:
sentences.head()

,sent
2,especially if you have to prepare a presentation
6,the meetings might be better if held in a roun...
9,plus your thoughts on any changes that need to...
11,i have been involved in most of the meetings a...
12,we have had three meetings which brought out v...


In [63]:
sentences.shape

(6429, 1)

<h1>2. Data Featurization

In [0]:
def generate_input_and_target(text):
    """
    Generates the input and target features from a single sentence
    """
    X = []
    y = []
    a = text.split()
    for i in range(1,len(a)):
        X.append(' '.join(a[:i]))
        y.append(' '.join(a[i:]))

    #Convert to dataframe
    b = pd.DataFrame(data={'X':X,'y':y})
    #Prepare input sequence to model, add start(#) and end tokens(*)
    b['X'] = '<start> ' + b['X'] + ' <end>'
    b['y'] = '<start> ' + b['y'] + ' <end>'
    
    return b

In [0]:
#Generate data for all samples
def generate_data_for_model(data_df):
    
    """
    Generates data for input to model
    """
    for i in tqdm(range(data_df.shape[0])):
        if i==0:
            
            final_df = generate_input_and_target(data_df.sent.iloc[0])
        else:
            final_df = pd.concat([final_df,generate_input_and_target(data_df.sent.iloc[i])],ignore_index=True)
    final_df = final_df.drop_duplicates()
    
    #Create column which counts number of words in each text
    final_df['X_length'] = final_df['X'].apply(lambda text:len(text.split()))

    #Drop samples with words less than 14 
    final_df = final_df[final_df['X_length']>14]
    
    return final_df[['X','y']]

In [102]:
data_final = generate_data_for_model(sentences)

In [103]:
data_final.head()

,X,y
41,<start> i have been involved in most of the me...,<start> been good <end>
42,<start> i have been involved in most of the me...,<start> good <end>
55,<start> we have had three meetings which broug...,<start> traders <end>
108,<start> please see the attached spreadsheet fo...,<start> summary <end>
194,<start> it would also be good to have each per...,<start> the event we need to reach them <end>


In [104]:
data_final.shape

(2272, 2)

In [0]:
[_,_,_,_,inp_embedding_matrix,out_embedding_matrix,inp_tokenizer,out_tokenizer] = joblib.load('encoded_data')

<h1>3. Modelling

<h2>3.1. Encoder Layer

In [0]:
#Encoder class
class Encoder(tf.keras.layers.Layer):
    def __init__(self, vocab_size, embedding_dim,embedding_matrix,enc_units,**kwargs):
        super(Encoder,self).__init__(**kwargs)
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.enc_units = enc_units
        self.embedding = Embedding(input_dim=self.vocab_size, output_dim=self.embedding_dim, weights=[embedding_matrix],
                                   mask_zero=True,name='Encoder_Embedding',trainable=False)
        self.lstm = LSTM(self.enc_units,return_sequences=True,return_state=True,name='Encoder_LSTM',dropout=0.33)

    def call(self, x):
        x = self.embedding(x)
        
        #state_c = Cell state output of last time step
        encoder_outputs, _, state_c = self.lstm(x)

        #Take average of encoder_outputs
        hidden = tf.reduce_mean(encoder_outputs,1)
        
        return hidden, state_c

<h2>3.2. Decoder Layer

In [0]:
class Decoder(tf.keras.layers.Layer):
    def __init__(self, vocab_size, embedding_dim,embedding_matrix,dec_units,**kwargs):
        super(Decoder, self).__init__(**kwargs)
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(input_dim=self.vocab_size, output_dim=self.embedding_dim,
                                            weights=[embedding_matrix],mask_zero=True,name='Decoder_Embedding',trainable=False)
        self.lstm = tf.keras.layers.LSTM(self.dec_units,return_sequences=True,return_state=True,name='Decoder_LSTM',
                                         dropout=0.33)
        self.batch_norm = BatchNormalization()
        
    def call(self, x, hidden, cell):
        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)
        
        # passing the concatenated vector to the LSTM
        lstm_output, state_h, state_c = self.lstm(x,initial_state=[hidden,cell])
        
        #Normalise output
        lstm_output = self.batch_norm(lstm_output)
        
        return lstm_output,state_h,state_c

<h2>3.3. Encoder Decoder Model

In [0]:
class Encoder_Decoder_Model(tf.keras.models.Model):
    def __init__(self, inp_vocab_size,out_vocab_size,embedding_dim,inp_embedding_matrix,out_embedding_matrix,units,**kwargs):
        super(Encoder_Decoder_Model,self).__init__(**kwargs)
        self.encoder = Encoder(vocab_size=inp_vocab_size, embedding_dim=embedding_dim,embedding_matrix=inp_embedding_matrix,
                               enc_units=units,name='Encoder_layer')
        self.decoder = Decoder(vocab_size=out_vocab_size, embedding_dim=embedding_dim,embedding_matrix=out_embedding_matrix,
                               dec_units=units,name='Decoder_layer')
        self.drop = Dropout(0.2,name='Dropout')
        self.dense = Dense(out_vocab_size,name='Output_layer')
        
    def call(self, inputs):
        #Inputs will contain encoder input and decoder input
        #Separate encoder and decoder inputs
        encoder_inputs, decoder_inputs = inputs[0], inputs[1]
        
        #Genearte output and hidden states from encoder object
        enc_hidden_output, enc_cell_output = self.encoder(encoder_inputs)

        #Generate output from decoder
        #Initialise hidden states of decoder with hidden states of encoder
        decoder_outputs,_,_ = self.decoder(decoder_inputs,enc_hidden_output,enc_cell_output)

        #Dropout
        decoder_outputs = self.drop(decoder_outputs)
        
        #Shape of outputs = (batch_size,timesteps,vocab_size)
        outputs = self.dense(decoder_outputs)
        
        return outputs

In [0]:
#Set Parameters
inp_vocab_size = len(inp_tokenizer.word_index) + 1  
out_vocab_size = len(out_tokenizer.word_index) + 1 
#Encoding and decoding Embedding layer dimension
embedding_dim = 300
#Encoding and decoding LSTM layer units
units = 256

#Initialise object for Model class
model = Encoder_Decoder_Model(inp_vocab_size,out_vocab_size,embedding_dim,inp_embedding_matrix,
                              out_embedding_matrix,units,name='Encoder_Decoder_Model')

In [76]:
#Max sequence length for input = max_length
#Max sequence length for output = max_length-1, 1 is subtracted because <end> token should not be give input to decoder
model.build(input_shape=[(None,23),(None,22)])
model.summary()

Model: "Encoder_Decoder_Model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Encoder_layer (Encoder)      multiple                  1011368   
_________________________________________________________________
Decoder_layer (Decoder)      multiple                  1012092   
_________________________________________________________________
Dropout (Dropout)            multiple                  0         
_________________________________________________________________
Output_layer (Dense)         multiple                  377533    
Total params: 2,400,993
Trainable params: 1,518,781
Non-trainable params: 882,212
_________________________________________________________________


In [0]:
#Load weights
model.load_weights('best_model.h5')

<h2>3.6. Predictions

In [0]:
def generate_output(enc_dec_model,input_sentence,add_tokens=False):
    
    input_max_length = 23
    output_max_length = 23
    
    #Add start and end tokens
    if add_tokens:
        input_sentence = '<start> ' + input_sentence + ' <end>'
    
    #Convert texts to sequences
    sent = inp_tokenizer.texts_to_sequences([input_sentence])
    #Pad data
    sent = tf.keras.preprocessing.sequence.pad_sequences(sent,maxlen=input_max_length,padding='post')
    
    #Get hidden states from encoder
    enc_hidden,enc_cell = enc_dec_model.layers[0](sent)
    
    #Boundary case for decoder
    dec_input = tf.expand_dims([out_tokenizer.word_index['<start>']], 1)
    dec_hidden = enc_hidden
    dec_cell = enc_cell
    output_sent = ''
    
    for i in range(output_max_length-1):
        #Get prediction from onestep_decoder
        dec_output,dec_hidden,dec_cell = enc_dec_model.layers[1](dec_input,dec_hidden,dec_cell)
        
        #Get prediction from dense layer
        #Shape == (batch_size,timestep,vocab_size) == (1,1,vocab_size)
        outputs = enc_dec_model.layers[3](dec_output)
        
        #Extract predicted id from decoder output
        key = np.argmax(outputs.numpy().reshape(-1))
        
        #Get word corresponding to index
        output_sent+=out_tokenizer.index_word[key]+' '
        
        if out_tokenizer.index_word[key] == '<end>':
            return output_sent
        
        #Make current decoder output as decoder input for next time step
        dec_input = tf.expand_dims([key], 0)
    
    return output_sent

In [0]:
def get_bleu_score(actual,prediction):
    prediction = '<start> ' + prediction
    return bleu_score.sentence_bleu([actual.split()],prediction.split())

In [83]:
joblib.dump([vocab,inp_embedding_matrix,out_embedding_matrix,inp_tokenizer,out_tokenizer],'utilties')

['utilties']

<h1>4. Final Functions

<h2>4.1. Final Function 1

In [0]:
def final_fun_1(dataset,perform_feat=True,add_tokens=False):
    """
    This function returns the predictions for input dataset
    
    Following files are needed in directory to run this function:
        1. vocab - Vocabulary of words on which model is trained
        2. inp_embedding_matrix - Weight matrix for Encoder Embedding layer
        3. dec_embedding_matrix - Weight matrix for Decoder Embedding layer
        4. inp_tokenizer - Tokenizer for encoder input
        5. out_tokenizer - Tokenizer for decoder output
        6. best_model.h5 - Weights of trained Encoder Decoder model
    
    Files from 1 to 5 are stored in'utilities'
    """
    start = time.time()
    #Load utilities
    [vocab,inp_embedding_matrix,out_embedding_matrix,inp_tokenizer,out_tokenizer] = joblib.load('utilties')
    
    if perform_feat:
        print('\n1. Sentence Extraction:')
        print('\n1.1. Extract messages:')
        messages = [email.message_from_string(i) for i in tqdm(dataset['message'])]

        print('\n1.2. Get content from messages:')
        #Get content from email
        dataset['content'] = [get_content(i) for i in tqdm(messages)]
        dataset = dataset['content'].to_frame()

        print('\n1.3. Remove insignificant emails:')
        #Remove insignificant emails
        dataset = dataset[~dataset['content'].str.contains('Outlook Migration Team@ENRON')]
        dataset = dataset[~dataset['content'].str.contains('Trade Counts and Volume')]

        print('\n1.4. Extract content from forwarded emails:')
        tqdm.pandas()
        dataset['content'] = dataset['content'].progress_apply(get_content_from_emails)

        print('\n1.5. Extract sentences:')
        sentences = sentence_df(dataset)

        print('\n1.6. Consider sentences which are in vocabulary:')
        indices = []
        for i in tqdm(range(sentences.shape[0])):
            for j in sentences.sent.iloc[i].split():
                if j not in vocab:
                    break
            else:
                indices.append(i)
        sentences = sentences.iloc[indices].drop_duplicates()

        print('\n2. Data Featurization:')
        #Featurize the data
        data_final = generate_data_for_model(sentences)
    else:
        data_final = dataset

    print('\n3. Modelling:')
    print('\n3.1. Build Encoder Decoder model:')
    #Set Parameters
    inp_vocab_size = len(inp_tokenizer.word_index) + 1  
    out_vocab_size = len(out_tokenizer.word_index) + 1 
    #Encoding and decoding Embedding layer dimension
    embedding_dim = 300
    #Encoding and decoding LSTM layer units
    units = 256
    #Initialise object for Model class
    model = Encoder_Decoder_Model(inp_vocab_size,out_vocab_size,embedding_dim,inp_embedding_matrix,
                              out_embedding_matrix,units,name='Encoder_Decoder_Model')
    model.build(input_shape=[(None,23),(None,22)])
    
    print('\n3.2. Load trained weights for the model:')
    model.load_weights('best_model.h5')
    
    print('\n3.3. Get predictions from model:')
    data_final['y_pred'] = None    
    for i in tqdm(range(data_final.shape[0])):
        prediction = generate_output(model,data_final.X.iloc[i],add_tokens=add_tokens)
        data_final['y_pred'].iloc[i] = '<start> '+prediction
    
    print('Time taken: {} minutes'.format(np.round((time.time()-start)/60,3)))

    return data_final[['X','y_pred']]

In [105]:
#Read the dataset
data = pd.read_csv('emails.csv',nrows=10000)
data.head()

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [106]:
#Make predictions from original dataset
preds = final_fun_1(data)


1. Sentence Extraction:

1.1. Extract messages:




1.2. Get content from messages:




1.3. Remove insignificant emails:

1.4. Extract content from forwarded emails:




1.5. Extract sentences:




1.6. Consider sentences which are in vocabulary:




2. Data Featurization:




3. Modelling:

3.1. Build Encoder Decoder model:

3.2. Load trained weights for the model:

3.3. Get predictions from model:



Time taken: 0.727 minutes


In [107]:
preds.head()

,X,y_pred
41,<start> i have been involved in most of the me...,<start> been made <end>
42,<start> i have been involved in most of the me...,<start> made <end>
55,<start> we have had three meetings which broug...,<start> issues <end>
108,<start> please see the attached spreadsheet fo...,<start> few issues <end>
194,<start> it would also be good to have each per...,<start> the office <end>


In [88]:
#Performs predictions when input sentences are given
preds2 = final_fun_1(data_final[['X']].sample(n=5000,replace=False),perform_feat=False,add_tokens=False)


3. Modelling:

3.1. Build Encoder Decoder model:

3.2. Load trained weights for the model:

3.3. Get predictions from model:



Time taken: 3.32 minutes


In [89]:
preds2.head()

,X,y_pred
33978,<start> you will be hearing more about this in...,<start> next few days <end>
1870,<start> i will do my best to provide answers a...,<start> on <end>
9427,<start> when you call give your name the numbe...,<start> the questions <end>
23699,<start> these numbers will be sent out as soon...,<start> are available <end>
48075,<start> i would like to work with you on a res...,<start> in the next few days <end>


In [0]:
def single_input_df(text):
    return pd.DataFrame(data={'X':[text]})

In [91]:
#Perform predictions when single sentence is given
inp_sentence = 'thank you'
preds3 = final_fun_1(single_input_df(inp_sentence),perform_feat=False,add_tokens=True)


3. Modelling:

3.1. Build Encoder Decoder model:

3.2. Load trained weights for the model:

3.3. Get predictions from model:



Time taken: 0.039 minutes


In [92]:
preds3

,X,y_pred
0,thank you,<start> for your time and consideration <end>


<h2>4.2. Final Function 2

In [0]:
def final_fun_2(dataset,perform_feat=True,add_tokens=False):
    """
    This function returns the bleu score
    
    Following files are needed in directory to run this function:
        1. vocab - Vocabulary of words on which model is trained
        2. inp_embedding_matrix - Weight matrix for Encoder Embedding layer
        3. dec_embedding_matrix - Weight matrix for Decoder Embedding layer
        4. inp_tokenizer - Tokenizer for encoder input
        5. out_tokenizer - Tokenizer for decoder output
        6. best_model.h5 - Weights of trained Encoder Decoder model
    
    Files from 1 to 5 are stored in'utilities'
    """
    start = time.time()
    #Load utilities
    [vocab,inp_embedding_matrix,out_embedding_matrix,inp_tokenizer,out_tokenizer] = joblib.load('utilties')
    
    if perform_feat:
        print('\n1. Sentence Extraction:')
        print('\n1.1. Extract messages:')
        messages = [email.message_from_string(i) for i in tqdm(dataset['message'])]

        print('\n1.2. Get content from messages:')
        #Get content from email
        dataset['content'] = [get_content(i) for i in tqdm(messages)]
        dataset = dataset['content'].to_frame()

        print('\n1.3. Remove insignificant emails:')
        #Remove insignificant emails
        dataset = dataset[~dataset['content'].str.contains('Outlook Migration Team@ENRON')]
        dataset = dataset[~dataset['content'].str.contains('Trade Counts and Volume')]

        print('\n1.4. Extract content from forwarded emails:')
        tqdm.pandas()
        dataset['content'] = dataset['content'].progress_apply(get_content_from_emails)

        print('\n1.5. Extract sentences:')
        sentences = sentence_df(dataset)

        print('\n1.6. Consider sentences which are in vocabulary:')
        indices = []
        for i in tqdm(range(sentences.shape[0])):
            for j in sentences.sent.iloc[i].split():
                if j not in vocab:
                    break
            else:
                indices.append(i)
        sentences = sentences.iloc[indices].drop_duplicates()

        print('\n2. Data Featurization:')
        #Featurize the data
        data_final = generate_data_for_model(sentences)
    else:
        data_final = dataset

    print('\n3. Modelling:')
    print('\n3.1. Build Encoder Decoder model:')
    #Set Parameters
    inp_vocab_size = len(inp_tokenizer.word_index) + 1  
    out_vocab_size = len(out_tokenizer.word_index) + 1 
    #Encoding and decoding Embedding layer dimension
    embedding_dim = 300
    #Encoding and decoding LSTM layer units
    units = 256
    #Initialise object for Model class
    model = Encoder_Decoder_Model(inp_vocab_size,out_vocab_size,embedding_dim,inp_embedding_matrix,
                              out_embedding_matrix,units,name='Encoder_Decoder_Model')
    model.build(input_shape=[(None,23),(None,22)])
    
    print('\n3.2. Load trained weights for the model:')
    model.load_weights('best_model.h5')
    
    print('\n3.3. Get predictions from model:')
    data_final['y_pred'] = None; data_final['bleu_score'] = None    
    for i in tqdm(range(data_final.shape[0])):
        prediction = generate_output(model,data_final.X.iloc[i],add_tokens=add_tokens)
        data_final['y_pred'].iloc[i] = '<start> '+prediction
        data_final['bleu_score'].iloc[i] = get_bleu_score(data_final.y.iloc[i],prediction)
    
    print('Time taken: {} minutes'.format(np.round((time.time()-start)/60,3)))

    return data_final.bleu_score.mean()

In [108]:
#Read the dataset
data = pd.read_csv('emails.csv',nrows=10000)
data.head()

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [109]:
#Get bleu_score from original dataset
bleu_score1 = final_fun_2(data)


1. Sentence Extraction:

1.1. Extract messages:




1.2. Get content from messages:




1.3. Remove insignificant emails:

1.4. Extract content from forwarded emails:




1.5. Extract sentences:




1.6. Consider sentences which are in vocabulary:




2. Data Featurization:




3. Modelling:

3.1. Build Encoder Decoder model:

3.2. Load trained weights for the model:

3.3. Get predictions from model:



Time taken: 0.722 minutes


In [110]:
print('BLEU Score for original dataset:',bleu_score1)

BLEU Score for original dataset: 0.692816756997191


In [113]:
#Get bleu_score when input sentences are given
bleu_score2 = final_fun_2(data_final[['X','y']].sample(n=2000,replace=False),perform_feat=False,add_tokens=False)


3. Modelling:

3.1. Build Encoder Decoder model:

3.2. Load trained weights for the model:

3.3. Get predictions from model:



Time taken: 1.292 minutes


In [114]:
print('BLEU Score for input sentences:',bleu_score2)

BLEU Score for input sentences: 0.7051859665367614
